In [1]:
!pip install flask-ngrok

In [3]:
!pip install surprise

     |████████████████████████████████| 11.8MB 7.7MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp36-cp36m-linux_x86_64.whl size=1670924 sha256=e9a11266a7a78ceace66bed6021cebb880e260c1889805ff1503bde725346d93
  Stored in directory: /root/.cache/pip/wheels/78/9c/3d/41b419c9d2aff5b6e2b4c0fc8d25c538202834058f9ed110d0
Successfully built scikit-surprise


In [4]:
import pandas as pd
import pickle
import requests
import json
from flask import Flask, request, jsonify
import surprise
from flask_ngrok import run_with_ngrok
from flask import Flask

In [16]:
app = Flask(__name__)
run_with_ngrok(app)
movie_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/movie_lens/movie.csv')
rating_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/movie_lens/rating.csv')

combined_small = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/movie_lens/combined_small.csv')
model = surprise.dump.load('/content/drive/MyDrive/Colab Notebooks/movie_lens/my_model')
predictions = model[0]
print(predictions[0])
#create index item name and value item id series
movie_rating_combined = pd.merge(movie_df,rating_df,on='movieId')
movie_info_small = movie_rating_combined.iloc[:5000000]
movies_list_id = pd.Series(movie_info_small['movieId'].unique(),
                            index=movie_info_small['title'].unique())



@app.route('/api',methods=['GET'])
def get_movies_recommendation():
  item_id = []
  movie_name = request.args.get('title')
  print('BOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO',movie_name)
  movie_id = movies_list_id.loc[movies_list_id.index == movie_name][0]
  user_id = combined_small.loc[combined_small.item == movie_id]['user'].tolist()[:5]
  for id in user_id:
    for i in range(len(predictions)):
      if predictions[i].uid == id:
        item_id.append(predictions[i].iid)
  movies_names = movies_list_id.loc[movies_list_id.isin(item_id)].index.tolist()[:5]
  return jsonify(movies_names)

if __name__=='__main__':
    app.run()
    

KeyboardInterrupt: ignored